# Neural Networks with PyTorch

In this assignment, we are going to train a Neural Networks on the Japanese MNIST dataset. It is composed of 70000 images of handwritten Hiragana characters. The target variables has 10 different classes.

Each image is of dimension 28 by 28. But we will flatten them to form a dataset composed of vectors of dimension (784, 1). The training process will be similar as for a structured dataset.

<img src='https://drive.google.com/uc?id=16TqEl9ESfXYbUpVafXD6h5UpJYGKfMxE' width="500" height="200">

Your goal is to run at least 3 experiments and get a model that can achieve 80% accuracy with not much overfitting on this dataset.

Some of the code have already been defined for you. You need only to add your code in the sections specified (marked with **TODO**). Some assert statements have been added to verify the expected outputs are correct. If it does throw an error, this means your implementation is behaving as expected.

Note: You can only use fully-connected and dropout layers for this assignment. You can not convolution layers for instance

# 1. Import Required Packages

[1.1] We are going to use numpy, matplotlib and google.colab packages

In [ ]:
from google.colab import drive
import numpy as np
import matplotlib.pyplot as plt

# 2. Download Dataset

We will store the dataset into your personal Google Drive.


[2.1] Mount Google Drive

In [ ]:
drive.mount('/content/gdrive')

[2.2] Create a folder called `DL_ASG_1` on your Google Drive at the root level

In [ ]:
! mkdir -p /content/gdrive/MyDrive/DL_ASG_1

[2.3] Navigate to this folder

In [ ]:
%cd '/content/gdrive/MyDrive/DL_ASG_1'

[2.4] Show the list of item on the folder

In [ ]:
!ls

[2.4] Dowload the dataset files to your Google Drive if required

In [ ]:
import requests
from tqdm import tqdm
import os.path

def download_file(url):
    path = url.split('/')[-1]
    if os.path.isfile(path):
        print (f"{path} already exists")
    else:
      r = requests.get(url, stream=True)
      with open(path, 'wb') as f:
          total_length = int(r.headers.get('content-length'))
          print('Downloading {} - {:.1f} MB'.format(path, (total_length / 1024000)))
          for chunk in tqdm(r.iter_content(chunk_size=1024), total=int(total_length / 1024) + 1, unit="KB"):
              if chunk:
                  f.write(chunk)

url_list = [
    'http://codh.rois.ac.jp/kmnist/dataset/kmnist/kmnist-train-imgs.npz',
    'http://codh.rois.ac.jp/kmnist/dataset/kmnist/kmnist-train-labels.npz',
    'http://codh.rois.ac.jp/kmnist/dataset/kmnist/kmnist-test-imgs.npz',
    'http://codh.rois.ac.jp/kmnist/dataset/kmnist/kmnist-test-labels.npz'
]

for url in url_list:
    download_file(url)

[2.5] List the content of the folder and confirm files have been dowloaded properly

In [ ]:
! ls

# 3. Load Data

[3.1] Import the required modules from PyTorch

In [ ]:
# TODO (Students need to fill this section)
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

[3.2] **TODO** Create 2 variables called `img_height` and `img_width` that will both take the value 28

In [ ]:
# TODO (Students need to fill this section)
img_height = 28
img_width = 28

[3.3] Create a function that loads a .npz file using numpy and return the content of the `arr_0` key

In [ ]:
def load(f):
    return np.load(f)['arr_0']

[3.4] **TODO** Load the 4 files saved on your Google Drive into their respective variables: x_train, y_train, x_test and y_test

In [ ]:
# TODO (Students need to fill this section)
x_train = load('kmnist-train-imgs.npz')
x_test = load('kmnist-test-imgs.npz')
y_train = load('kmnist-train-labels.npz')
y_test = load('kmnist-test-labels.npz')


[3.5] **TODO** Using matplotlib display the first image from the train set and its target value

In [ ]:
# TODO (Students need to fill this section)
plt.imshow(x_train[0])
print(y_train[0])

# 4. Prepare Data

[4.1] **TODO** Reshape the images from the training and testing set to have the channel dimension last. The dimensions should be: (row_number, height, width, channel)

In [ ]:
# TODO (Students need to fill this section)
x_train = x_train.reshape(x_train.shape[0], img_height, img_width, 1)
x_test = x_test.reshape(x_test.shape[0], img_height, img_width, 1)

In [ ]:
print(f"x_train shape: {x_train.shape}")
print(f"x_test shape: {x_test.shape}")

[4.2] **TODO** Cast `x_train` and `x_test` into `float32` decimals

In [ ]:
# TODO (Students need to fill this section)
x_train = x_train.astype(np.float32)
x_test = x_test.astype(np.float32)

[4.3] **TODO** Standardise the images of the training and testing sets. Originally each image contains pixels with value ranging from 0 to 255. after standardisation, the new value range should be from 0 to 1.

In [ ]:
# TODO (Students need to fill this section)
x_train = x_train / 255
x_test = x_test / 255


In [ ]:
print(f"Min value in x_train: {x_train.min()}, Max value in x_train: {x_train.max()}")
print(f"Min value in x_test: {x_test.min()}, Max value in x_test: {x_test.max()}")

In [ ]:
# Solution
print(x_train[0][0].shape)

[4.4] **TODO** Create a variable called `num_classes` that will take the value 10 which corresponds to the number of classes for the target variable

In [ ]:
# TODO (Students need to fill this section)
num_classes = 10

[4.5] **TODO** Convert the target variable for the training and testing sets to a binary class matrix of dimension (rows, num_classes).

For example:
- class 0 will become [1, 0, 0, 0, 0, 0, 0, 0, 0, 0]
- class 1 will become [0, 1, 0, 0, 0, 0, 0, 0, 0, 0]
- class 5 will become [0, 0, 0, 0, 0, 1, 0, 0, 0, 0]
- class 9 will become [0, 0, 0, 0, 0, 0, 0, 0, 0, 1]

In [ ]:
import numpy as np
print("Unique values in y_train:", np.unique(y_train))
print("Unique values in y_test:", np.unique(y_test))

In [ ]:
from tensorflow.keras.utils import to_categorical

num_classes = 10

y_train = to_categorical(y_train, num_classes)
y_test = to_categorical(y_test, num_classes)


print(f"y_train shape: {y_train.shape}")
print(f"y_test shape: {y_test.shape}")


In [ ]:
y_train[0]

In [ ]:
y_test[0]

## Splitting Data into Train and Val

In [ ]:
from sklearn.model_selection import train_test_split

x_train, x_val, y_train, y_val = train_test_split(x_train, y_train, test_size=0.1, random_state=42, stratify=y_train)

print(f"x_train shape: {x_train.shape}, y_train shape: {y_train.shape}")
print(f"x_val shape: {x_val.shape}, y_val shape: {y_val.shape}")


# 5. Experiment 1

## 5. Define Neural Networks Architecure

[5.1] Set the seed in PyTorch for reproducing results



In [ ]:
# TODO (Students need to fill this section)
torch.manual_seed(21)

[5.2] **TODO** Define the architecture of your Neural Networks and save it into a variable called `model`

In [ ]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'

In [ ]:
# TODO (Students need to fill this section)
layer_1 = nn.Linear(784, 512)
layer_2 = nn.Linear(512, 512)
layer_top = nn.Linear(512, 10)

In [ ]:
model1 = nn.Sequential(
    nn.Flatten(),
    layer_1,
    nn.ReLU(),
    layer_2,
    nn.ReLU(),
    layer_top
)

[5.2] **TODO** Print the summary of your model

In [ ]:
# TODO (Students need to fill this section)
print(model1)


In [ ]:
model1.to(device)

## 6. Train Neural Networks

[6.1] **TODO** Create 2 variables called `batch_size` and `epochs` that will  respectively take the values 128 and 500

In [ ]:
# TODO (Students need to fill this section)
batch_size = 128
epochs = 500

[6.2] **TODO** Compile your model with the appropriate loss function, the optimiser of your choice and the accuracy metric

In [ ]:
criterion = nn.CrossEntropyLoss()

In [ ]:
optimizer = optim.Adam(model1.parameters(), lr=0.001)

In [ ]:
import torch
from torch.utils.data import DataLoader, TensorDataset

# Detect GPU (CUDA) or fallback to CPU
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")

# Convert NumPy arrays to PyTorch tensors and move to the correct device
x_train_tensor = torch.tensor(x_train, dtype=torch.float32).to(device)
y_train_tensor = torch.tensor(y_train, dtype=torch.float32).to(device)

x_val_tensor = torch.tensor(x_val, dtype=torch.float32).to(device)
y_val_tensor = torch.tensor(y_val, dtype=torch.float32).to(device)

x_test_tensor = torch.tensor(x_test, dtype=torch.float32).to(device)
y_test_tensor = torch.tensor(y_test, dtype=torch.float32).to(device)

# Create Dataset objects
train_dataset = TensorDataset(x_train_tensor, y_train_tensor)
val_dataset = TensorDataset(x_val_tensor, y_val_tensor)
test_dataset = TensorDataset(x_test_tensor, y_test_tensor)

# Define batch size
BATCH_SIZE = batch_size  # Ensure batch_size is properly defined

# Create DataLoaders
dataloader_train = DataLoader(train_dataset, batch_size=BATCH_SIZE, shuffle=True)
dataloader_val = DataLoader(val_dataset, batch_size=BATCH_SIZE, shuffle=False)
dataloader_test = DataLoader(test_dataset, batch_size=BATCH_SIZE, shuffle=False)

# Check the number of batches
print(f"Train batches: {len(dataloader_train)}")
print(f"Validation batches: {len(dataloader_val)}")
print(f"Test batches: {len(dataloader_test)}")


[6.3] **TODO** Train your model
using the number of epochs defined. Calculate the total loss and save it to a variable called total_loss.

In [ ]:
EPOCHS = epochs

train_losses = []
val_losses = []

for epoch in range(EPOCHS):
    model1.train()
    train_loss = 0

    for x_batch, y_batch in dataloader_train:
        optimizer.zero_grad()
        outputs = model1(x_batch)
        loss = criterion(outputs, y_batch)
        loss.backward()
        optimizer.step()

        train_loss += loss.item()

    train_losses.append(train_loss / len(dataloader_train))

    model1.eval()
    val_loss = 0

    with torch.no_grad():
        for x_val, y_val in dataloader_val:
            val_outputs = model1(x_val)
            val_loss += criterion(val_outputs, y_val).item()

    val_losses.append(val_loss / len(dataloader_val))

    print(f"Epoch {epoch+1}/{EPOCHS}, Train Loss: {train_losses[-1]:.4f}, Val Loss: {val_losses[-1]:.4f}")


In [ ]:
total_train_loss = sum(train_losses)
final_train_loss = train_losses[-1]

total_val_loss = sum(val_losses)
final_val_loss = val_losses[-1]

print(f"Total Training Loss: {total_train_loss:.4f}")
print(f"Final Training Loss: {final_train_loss:.4f}")
print(f"Total Validation Loss: {total_val_loss:.4f}")
print(f"Final Validation Loss: {final_val_loss:.4f}")

[6.4] **TODO** Test your model.  Initiate the model.eval() along with torch.no_grad() to turn off the gradients.


In [ ]:
from sklearn.metrics import confusion_matrix
import torch

model1.eval()
correct = 0
total = 0
total_test_loss = 0

predicted_labels = []
true_labels = []

with torch.no_grad():
    for data, target in dataloader_test:
        data = data.view(-1, 28*28).to(device)
        target = target.to(device)

        if target.ndim > 1:
            target = target.argmax(dim=1)

        outputs = model1(data)
        loss = criterion(outputs, target)
        total_test_loss += loss.item()

        _, predicted = torch.max(outputs, 1)

        total += target.size(0)
        correct += (predicted == target).sum().item()

        predicted_labels.extend(predicted.cpu().tolist())
        true_labels.extend(target.cpu().tolist())

avg_test_loss = total_test_loss / len(dataloader_test)

print(f"Test Loss: {avg_test_loss:.4f}")



In [ ]:
from sklearn.metrics import confusion_matrix
import torch

def evaluate_model(model, dataloader, criterion, device, dataset_type="test"):
    model.eval()
    correct = 0
    total = 0
    total_loss = 0

    predicted_labels = []
    true_labels = []

    with torch.no_grad():
        for data, target in dataloader:
            data = data.view(-1, 28*28).to(device)
            target = target.to(device)

            if target.ndim > 1:
                target = target.argmax(dim=1)

            outputs = model(data)
            loss = criterion(outputs, target)
            total_loss += loss.item()

            _, predicted = torch.max(outputs, 1)

            total += target.size(0)
            correct += (predicted == target).sum().item()

            predicted_labels.extend(predicted.cpu().tolist())
            true_labels.extend(target.cpu().tolist())

    avg_loss = total_loss / len(dataloader)

    accuracy = correct / total

    print(f"{dataset_type.capitalize()} Loss: {avg_loss:.4f}")
    print(f"{dataset_type.capitalize()} Accuracy: {accuracy:.4f}")

    return avg_loss, accuracy


In [ ]:
val_loss, val_accuracy = evaluate_model(model1, dataloader_val, criterion, device, dataset_type="val")


In [ ]:
train_loss, train_accuracy = evaluate_model(model1, dataloader_train, criterion, device, dataset_type="train")


## 7. Analyse Results

[7.1] **TODO** Display the performance of your model on the training and testing sets

In [ ]:
print (outputs[0])

In [ ]:
pred_test = np.argmax(outputs.cpu().numpy(), axis=1)
pred_test[0]

In [ ]:
target[0]

In [ ]:
test_loss, test_accuracy = evaluate_model(model1, dataloader_test, criterion, device, dataset_type="test")


In [ ]:
accuracy = correct / total
print(f"Test Accuracy: {accuracy * 100:.2f}%")

[7.2] **TODO** Plot the learning curve of your model

In [ ]:
import matplotlib.pyplot as plt

plt.plot(train_losses, label='Train Loss')
plt.plot(val_losses, label='Validation Loss')
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.legend()
plt.title('Train vs Validation Loss')
plt.show()


[7.3] **TODO** Display the confusion matrix on the testing set predictions

In [ ]:
conf_matrix = confusion_matrix(true_labels, predicted_labels)
import matplotlib.pyplot as plt
import seaborn as sns
plt.figure(figsize=(10, 8))
sns.heatmap(conf_matrix, annot=True, fmt='d', cmap='Blues', cbar=False)
plt.xlabel('Predicted labels')
plt.ylabel('True labels')
plt.title('Confusion Matrix')
plt.show()

# Experiment 2

## 5. Define Neural Networks Architecure

[5.1] Set the seed in PyTorch for reproducing results



In [ ]:
# TODO (Students need to fill this section)
torch.manual_seed(44)

[5.2] **TODO** Define the architecture of your Neural Networks and save it into a variable called `model`

In [ ]:
# TODO (Students need to fill this section)
layer_1 = nn.Linear(784, 512)
layer_2 = nn.Linear(512, 512)
layer_top = nn.Linear(512, 10)

In [ ]:
model2 = nn.Sequential(
    nn.Flatten(),
    layer_1,
    nn.ReLU(),
    nn.Dropout(p=0.3),
    layer_2,
    nn.ReLU(),
    nn.Dropout(p=0.3),
    layer_top
)

[5.2] **TODO** Print the summary of your model

In [ ]:
# TODO (Students need to fill this section)
print(model2)


In [ ]:
model2.to(device)

## 6. Train Neural Networks

[6.1] **TODO** Create 2 variables called `batch_size` and `epochs` that will  respectively take the values 128 and 500

In [ ]:
# TODO (Students need to fill this section)
batch_size = 128
epochs = 20

[6.2] **TODO** Compile your model with the appropriate loss function, the optimiser of your choice and the accuracy metric

In [ ]:
criterion = nn.CrossEntropyLoss()

In [ ]:
optimizer = optim.Adam(model2.parameters(), lr=0.001)

In [ ]:
model2.to(device)

[6.3] **TODO** Train your model
using the number of epochs defined. Calculate the total loss and save it to a variable called total_loss.

In [ ]:
EPOCHS = epochs

train_losses = []
val_losses = []

for epoch in range(EPOCHS):
    model2.train()
    train_loss = 0

    for x_batch, y_batch in dataloader_train:
        optimizer.zero_grad()
        outputs = model2(x_batch)
        loss = criterion(outputs, y_batch)
        loss.backward()
        optimizer.step()

        train_loss += loss.item()

    train_losses.append(train_loss / len(dataloader_train))

    model2.eval()
    val_loss = 0

    with torch.no_grad():
        for x_val, y_val in dataloader_val:
            val_outputs = model2(x_val)
            val_loss += criterion(val_outputs, y_val).item()

    val_losses.append(val_loss / len(dataloader_val))

    print(f"Epoch {epoch+1}/{EPOCHS}, Train Loss: {train_losses[-1]:.4f}, Val Loss: {val_losses[-1]:.4f}")


[6.4] **TODO** Test your model.  Initiate the model.eval() along with torch.no_grad() to turn off the gradients.


In [ ]:
from sklearn.metrics import confusion_matrix
import torch

model2.eval()
correct = 0
total = 0
total_test_loss = 0

predicted_labels = []
true_labels = []

with torch.no_grad():
    for data, target in dataloader_test:
        data = data.view(-1, 28*28).to(device)
        target = target.to(device)

        if target.ndim > 1:
            target = target.argmax(dim=1)

        outputs = model2(data)
        loss = criterion(outputs, target)
        total_test_loss += loss.item()

        _, predicted = torch.max(outputs, 1)

        total += target.size(0)
        correct += (predicted == target).sum().item()

        predicted_labels.extend(predicted.cpu().tolist())
        true_labels.extend(target.cpu().tolist())

avg_test_loss = total_test_loss / len(dataloader_test)

print(f"Test Loss: {avg_test_loss:.4f}")



In [ ]:
val_loss, val_accuracy = evaluate_model(model2, dataloader_val, criterion, device, dataset_type="val")
train_loss, train_accuracy = evaluate_model(model2, dataloader_train, criterion, device, dataset_type="train")

## 7. Analyse Results

[7.1] **TODO** Display the performance of your model on the training and testing sets

In [ ]:
total_train_loss = sum(train_losses)


total_val_loss = sum(val_losses)


print(f"Total Training Loss: {total_train_loss:.4f}")

print(f"Total Validation Loss: {total_val_loss:.4f}")


In [ ]:
print (outputs[0])

In [ ]:
pred_test = np.argmax(outputs.cpu().numpy(), axis=1)
pred_test[0]

In [ ]:
target[0]

In [ ]:
test_loss, test_accuracy = evaluate_model(model2, dataloader_test, criterion, device, dataset_type="test")


[7.2] **TODO** Plot the learning curve of your model

In [ ]:
import matplotlib.pyplot as plt

plt.figure(figsize=(8,6))
plt.plot(range(1, len(train_losses) + 1), train_losses, label="Training Loss")
plt.plot(range(1, len(train_losses) + 1), val_losses, label="Training Loss")
plt.xlabel("Epochs")
plt.ylabel("Loss")
plt.title("Learning Curve")
plt.legend()
plt.grid(True)
plt.show()


[7.3] **TODO** Display the confusion matrix on the testing set predictions

In [ ]:
conf_matrix = confusion_matrix(true_labels, predicted_labels)
# Solution
import matplotlib.pyplot as plt
import seaborn as sns
plt.figure(figsize=(10, 8))
sns.heatmap(conf_matrix, annot=True, fmt='d', cmap='Blues', cbar=False)
plt.xlabel('Predicted labels')
plt.ylabel('True labels')
plt.title('Confusion Matrix')
plt.show()

# Experiment 3


## 5. Define Neural Networks Architecure

[5.1] Set the seed in PyTorch for reproducing results



In [ ]:
# TODO (Students need to fill this section)
torch.manual_seed(46)

[5.2] **TODO** Define the architecture of your Neural Networks and save it into a variable called `model`

In [ ]:
# TODO (Students need to fill this section)
layer_1 = nn.Linear(784, 512)
layer_2 = nn.Linear(512, 512)
layer_3 = nn.Linear(512, 216)
layer_top = nn.Linear(216, 10)

In [ ]:
model3 = nn.Sequential(
    nn.Flatten(),
    layer_1,
    nn.ReLU(),
    nn.Dropout(p=0.3),
    layer_2,
    nn.ReLU(),
    nn.Dropout(p=0.3),
    layer_3,
    nn.ReLU(),
    nn.Dropout(p=0.3),
    layer_top
)

[5.2] **TODO** Print the summary of your model

In [ ]:
# TODO (Students need to fill this section)
print(model3)


In [ ]:
model3.to(device)

## 6. Train Neural Networks

[6.1] **TODO** Create 2 variables called `batch_size` and `epochs` that will  respectively take the values 128 and 500

In [ ]:
# TODO (Students need to fill this section)
batch_size = 128
epochs = 10

[6.2] **TODO** Compile your model with the appropriate loss function, the optimiser of your choice and the accuracy metric

In [ ]:
criterion = nn.CrossEntropyLoss()

In [ ]:
optimizer = optim.Adam(model3.parameters(), lr=0.001)

In [ ]:
model3.to(device)

[6.3] **TODO** Train your model
using the number of epochs defined. Calculate the total loss and save it to a variable called total_loss.

In [ ]:
EPOCHS = epochs

train_losses = []
val_losses = []

for epoch in range(EPOCHS):
    model3.train()
    train_loss = 0

    for x_batch, y_batch in dataloader_train:
        optimizer.zero_grad()
        outputs = model3(x_batch)
        loss = criterion(outputs, y_batch)
        loss.backward()
        optimizer.step()

        train_loss += loss.item()

    train_losses.append(train_loss / len(dataloader_train))

    model3.eval()
    val_loss = 0

    with torch.no_grad():
        for x_val, y_val in dataloader_val:
            val_outputs = model3(x_val)
            val_loss += criterion(val_outputs, y_val).item()

    val_losses.append(val_loss / len(dataloader_val))

    print(f"Epoch {epoch+1}/{EPOCHS}, Train Loss: {train_losses[-1]:.4f}, Val Loss: {val_losses[-1]:.4f}")


[6.4] **TODO** Test your model.  Initiate the model.eval() along with torch.no_grad() to turn off the gradients.


In [ ]:
from sklearn.metrics import confusion_matrix
import torch

model3.eval()
correct = 0
total = 0
total_test_loss = 0

predicted_labels = []
true_labels = []

with torch.no_grad():
    for data, target in dataloader_test:
        data = data.view(-1, 28*28).to(device)
        target = target.to(device)

        if target.ndim > 1:
            target = target.argmax(dim=1)

        outputs = model3(data)
        loss = criterion(outputs, target)
        total_test_loss += loss.item()

        _, predicted = torch.max(outputs, 1)

        total += target.size(0)
        correct += (predicted == target).sum().item()

        predicted_labels.extend(predicted.cpu().tolist())
        true_labels.extend(target.cpu().tolist())

avg_test_loss = total_test_loss / len(dataloader_test)

print(f"Test Loss: {avg_test_loss:.4f}")



In [ ]:
val_loss, val_accuracy = evaluate_model(model3, dataloader_val, criterion, device, dataset_type="val")


In [ ]:
train_loss, train_accuracy = evaluate_model(model3, dataloader_train, criterion, device, dataset_type="train")


## 7. Analyse Results

[7.1] **TODO** Display the performance of your model on the training and testing sets

In [ ]:
total_train_loss = sum(train_losses)
final_train_loss = train_losses[-1]
total_val_loss = sum(val_losses)
final_val_loss = val_losses[-1]

print(f"Total Training Loss: {total_train_loss:.4f}")
print(f"Final Training Loss: {final_train_loss:.4f}")
print(f"Total Validation Loss: {total_val_loss:.4f}")
print(f"Final Validation Loss: {final_val_loss:.4f}")

In [ ]:
print (outputs[0])

In [ ]:
pred_test = np.argmax(outputs.cpu().numpy(), axis=1)
pred_test[0]

In [ ]:
target[0]

In [ ]:
accuracy = correct / total
print(f"Test Accuracy: {accuracy * 100:.2f}%")

In [ ]:
test_loss, test_accuracy = evaluate_model(model3, dataloader_test, criterion, device, dataset_type="test")


In [ ]:
test_loss, test_accuracy = evaluate_model(model3, dataloader_test, criterion, device, dataset_type="test")


[7.2] **TODO** Plot the learning curve of your model

In [ ]:
import matplotlib.pyplot as plt

plt.figure(figsize=(8,6))
plt.plot(range(1, len(train_losses) + 1), train_losses, label="Training Loss")
plt.plot(range(1, len(train_losses) + 1), val_losses, label="Training Loss")
plt.xlabel("Epochs")
plt.ylabel("Loss")
plt.title("Learning Curve")
plt.legend()
plt.grid(True)
plt.show()


[7.3] **TODO** Display the confusion matrix on the testing set predictions

In [ ]:
conf_matrix = confusion_matrix(true_labels, predicted_labels)
import matplotlib.pyplot as plt
import seaborn as sns
plt.figure(figsize=(10, 8))
sns.heatmap(conf_matrix, annot=True, fmt='d', cmap='Blues', cbar=False)
plt.xlabel('Predicted labels')
plt.ylabel('True labels')
plt.title('Confusion Matrix')
plt.show()

# Experiment 4

## 4. Define Neural Networks Architecure

[5.1] Set the seed in PyTorch for reproducing results



In [ ]:
# TODO (Students need to fill this section)
torch.manual_seed(6)

[5.2] **TODO** Define the architecture of your Neural Networks and save it into a variable called `model`

In [ ]:
# TODO (Students need to fill this section)
layer_1 = nn.Linear(784, 512)
layer_2 = nn.Linear(512, 512)
layer_3 = nn.Linear(512, 256)
layer_top = nn.Linear(256, 10)

In [ ]:
model4 = nn.Sequential(
    nn.Flatten(),
    layer_1,
    nn.ReLU(),
    nn.Dropout(p=0.3),
    layer_2,
    nn.ReLU(),
    nn.Dropout(p=0.3),
    layer_3,
    nn.ReLU(),
    nn.Dropout(p=0.3),
    layer_top
)

[5.2] **TODO** Print the summary of your model

In [ ]:
# TODO (Students need to fill this section)
print(model4)


In [ ]:
model4.to(device)

## 6. Train Neural Networks

[6.1] **TODO** Create 2 variables called `batch_size` and `epochs` that will  respectively take the values 128 and 500

In [ ]:
# TODO (Students need to fill this section)
batch_size = 128
epochs = 10

[6.2] **TODO** Compile your model with the appropriate loss function, the optimiser of your choice and the accuracy metric

In [ ]:
criterion = nn.CrossEntropyLoss()

In [ ]:
optimizer = optim.Adam(model4.parameters(), lr=0.001, weight_decay = 0.001)

In [ ]:
model4.to(device)

[6.3] **TODO** Train your model
using the number of epochs defined. Calculate the total loss and save it to a variable called total_loss.

In [ ]:
EPOCHS = epochs

train_losses = []
val_losses = []

for epoch in range(EPOCHS):
    model4.train()
    train_loss = 0

    for x_batch, y_batch in dataloader_train:
        optimizer.zero_grad()
        outputs = model4(x_batch)
        loss = criterion(outputs, y_batch)
        loss.backward()
        optimizer.step()

        train_loss += loss.item()

    train_losses.append(train_loss / len(dataloader_train))

    model4.eval()
    val_loss = 0

    with torch.no_grad():
        for x_val, y_val in dataloader_val:
            val_outputs = model4(x_val)
            val_loss += criterion(val_outputs, y_val).item()

    val_losses.append(val_loss / len(dataloader_val))

    print(f"Epoch {epoch+1}/{EPOCHS}, Train Loss: {train_losses[-1]:.4f}, Val Loss: {val_losses[-1]:.4f}")


[6.4] **TODO** Test your model.  Initiate the model.eval() along with torch.no_grad() to turn off the gradients.


In [ ]:
from sklearn.metrics import confusion_matrix
import torch

model4.eval()
correct = 0
total = 0
total_test_loss = 0

predicted_labels = []
true_labels = []

with torch.no_grad():
    for data, target in dataloader_test:
        data = data.view(-1, 28*28).to(device)
        target = target.to(device)

        if target.ndim > 1:
            target = target.argmax(dim=1)

        outputs = model4(data)
        loss = criterion(outputs, target)
        total_test_loss += loss.item()

        _, predicted = torch.max(outputs, 1)

        total += target.size(0)
        correct += (predicted == target).sum().item()

        predicted_labels.extend(predicted.cpu().tolist())
        true_labels.extend(target.cpu().tolist())

avg_test_loss = total_test_loss / len(dataloader_test)

print(f"Test Loss: {avg_test_loss:.4f}")



## 7. Analyse Results

[7.1] **TODO** Display the performance of your model on the training and testing sets

In [ ]:
total_train_loss = sum(train_losses)
final_train_loss = train_losses[-1]

total_val_loss = sum(val_losses)
final_val_loss = val_losses[-1]

print(f"Total Training Loss: {total_train_loss:.4f}")
print(f"Final Training Loss: {final_train_loss:.4f}")
print(f"Total Validation Loss: {total_val_loss:.4f}")
print(f"Final Validation Loss: {final_val_loss:.4f}")

In [ ]:
print (outputs[0])

In [ ]:
pred_test = np.argmax(outputs.cpu().numpy(), axis=1)
pred_test[0]

In [ ]:
target[0]

In [ ]:
accuracy = correct / total
print(f"Test Accuracy: {accuracy * 100:.2f}%")

In [ ]:
test_loss, test_accuracy = evaluate_model(model4, dataloader_test, criterion, device, dataset_type="test")
val_loss, val_accuracy = evaluate_model(model4, dataloader_val, criterion, device, dataset_type="val")
train_loss, train_accuracy = evaluate_model(model4, dataloader_train, criterion, device, dataset_type="train")


[7.2] **TODO** Plot the learning curve of your model

In [ ]:
import matplotlib.pyplot as plt

plt.figure(figsize=(8,6))
plt.plot(range(1, len(train_losses) + 1), train_losses, label="Training Loss")
plt.plot(range(1, len(train_losses) + 1), val_losses, label="Training Loss")
plt.xlabel("Epochs")
plt.ylabel("Loss")
plt.title("Learning Curve")
plt.legend()
plt.grid(True)
plt.show()


[7.3] **TODO** Display the confusion matrix on the testing set predictions

In [ ]:
conf_matrix = confusion_matrix(true_labels, predicted_labels)
import matplotlib.pyplot as plt
import seaborn as sns
# Plotting the confusion matrix
plt.figure(figsize=(10, 8))
sns.heatmap(conf_matrix, annot=True, fmt='d', cmap='Blues', cbar=False)
plt.xlabel('Predicted labels')
plt.ylabel('True labels')
plt.title('Confusion Matrix')
plt.show()